In [55]:
import pandas as pd
import numpy as np
import math
from datetime import date
from datetime import datetime

#set file name  - Confirmed, Deaths, Recovered

In [97]:
filename = 'Confirmed'

In [98]:
dt = datetime.now().strftime('%Y%m%d')

Covid 19 cases data from CSSE

In [99]:
df = pd.read_csv('covid_data/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-' + filename + '.csv')

In [100]:
df.shape

(450, 58)

In [101]:
df.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,NaN,Japan,36.0000,138.0000,0,0,0,0,0,0,...,6,6,6,10,10,15,16,19,22,22
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
df1 = df[df['Country/Region'] == 'US']

In [103]:
df1.shape

(247, 58)

In [104]:
df2 = pd.melt(df1, id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'period', value_name = 'case_counts')

In [105]:
df3 = df2[df2['case_counts'] > 0]

In [106]:
df3.shape

(74, 6)

In [107]:
df3.to_csv('Clean_' + filename + '_' + dt + '.csv')

In [108]:
df3.head()

,Province/State,Country/Region,Lat,Long,period,case_counts
9576,"King County, WA",US,47.6062,-122.3321,2/29/20,1
9823,"King County, WA",US,47.6062,-122.3321,3/1/20,1
10064,"Snohomish County, WA",US,48.0330,-121.8339,3/2/20,1
10070,"King County, WA",US,47.6062,-122.3321,3/2/20,5
10311,"Snohomish County, WA",US,48.0330,-121.8339,3/3/20,1


'/Users/hong/Workspace/github/COVID-19'

In [109]:
zipmap = pd.read_csv('zipcodemap/us-zip-code-latitude-and-longitude.csv', sep = ';')

In [110]:
zipmap.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,71937,Cove,AR,34.398483,-94.39398,-6,1,"34.398483,-94.39398"
1,72044,Edgemont,AR,35.624351,-92.16056,-6,1,"35.624351,-92.16056"
2,56171,Sherburn,MN,43.660847,-94.74357,-6,1,"43.660847,-94.74357"
3,49430,Lamont,MI,43.010337,-85.89754,-5,1,"43.010337,-85.89754"
4,52585,Richland,IA,41.194129,-91.98027,-6,1,"41.194129,-91.98027"


zipcode Lat Long data from
https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/export/


/Users/hong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


new = zipmap1['geopoint'].str.split(',', n= 1, expand = True)

zipmap1['Long_z'] = new[0]

zipmap1['Lat_z'] = new[1]

In [70]:
zipmap['tmp'] = 1

In [71]:
df3['tmp'] = 1

/Users/hong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


####  join Case with Map

In [72]:
df4 = df3.merge(zipmap, on = 'tmp',how = 'outer')

In [73]:
df4['Country/Region'].value_counts()

US    45523314
Name: Country/Region, dtype: int64

In [74]:
df4.shape

(45523314, 15)

In [75]:
df4.head()

,Province/State,Country/Region,Lat,Long,period,case_counts,tmp,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,"King County, WA",US,47.6062,-122.3321,1/22/20,1,1,71937,Cove,AR,34.398483,-94.39398,-6,1,"34.398483,-94.39398"
1,"King County, WA",US,47.6062,-122.3321,1/22/20,1,1,72044,Edgemont,AR,35.624351,-92.16056,-6,1,"35.624351,-92.16056"
2,"King County, WA",US,47.6062,-122.3321,1/22/20,1,1,56171,Sherburn,MN,43.660847,-94.74357,-6,1,"43.660847,-94.74357"
3,"King County, WA",US,47.6062,-122.3321,1/22/20,1,1,49430,Lamont,MI,43.010337,-85.89754,-5,1,"43.010337,-85.89754"
4,"King County, WA",US,47.6062,-122.3321,1/22/20,1,1,52585,Richland,IA,41.194129,-91.98027,-6,1,"41.194129,-91.98027"


(38569563, 15)

df4['distance'] = math.sqrt(pow(df4['Lat'] - df4['Latitude'], 2) + pow(df4['Long'] - df4['Longitude'], 2))

In [76]:
df4['distance'] = pow(df4['Lat'] - df4['Latitude'], 2) + pow(df4['Long'] - df4['Longitude'], 2)

In [77]:
df5 = df4[df4['distance'] <2.89]

In [78]:
df5.shape

(858188, 16)

In [79]:
df4.shape

(45523314, 16)

In [80]:
df5.to_csv('zipaffected_' + filename + '_' + dt + '.csv')

### National Level

In [111]:
df_national = df3.groupby(['period'])['case_counts'].agg('sum')

In [112]:
df_national.to_csv(filename + "_" + dt + "_national.csv", header = True, index = True)

'Deaths'